In [1]:
# 1. General Libraries & Setup
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random 
from PIL import Image

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import preprocess_input

print("TensorFlow version:", tf.__version__)
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available:", len(physical_devices)) 

# Paths and constants
BASE_DIR = 'data'
TRAIN_DIR = os.path.join(BASE_DIR, 'train')
VAL_DIR = os.path.join(BASE_DIR, 'val')
TEST_DIR = os.path.join(BASE_DIR, 'test')

TRAIN_CSV = os.path.join(BASE_DIR, 'train.csv')
VAL_CSV = os.path.join(BASE_DIR, 'valid.csv')
TEST_CSV = os.path.join(BASE_DIR, 'test.csv')

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 42

/Users/nikhilgiji/Drive/cs/computer_vision/RetinoGrade/retino/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


TensorFlow version: 2.15.0
Num GPUs Available: 1


In [2]:
# 2. Load CSVs and EDA

train_df = pd.read_csv(TRAIN_CSV)
val_df = pd.read_csv(VAL_CSV)
test_df = pd.read_csv(TEST_CSV)

print("Train class distribution:")
print(train_df['diagnosis'].value_counts())

sns.countplot(data=train_df, x='diagnosis')
plt.title("Class Distribution in Training Set")
plt.xlabel("Diabetic Retinopathy Grade")
plt.ylabel("Image Count")
plt.show()

Train class distribution:
diagnosis
0    1444
2     799
1     296
4     236
3     154
Name: count, dtype: int64


In [3]:
# 3. Visualize Sample Images per Class

def show_sample_images(df, directory, label_col='diagnosis', image_col='id_code', samples_per_class=3):
    labels = sorted(df[label_col].unique())
    plt.figure(figsize=(15, 5 * len(labels)))

    for class_idx, label in enumerate(labels):
        sample_paths = df[df[label_col] == label].sample(samples_per_class, random_state=SEED)
        for i, (_, row) in enumerate(sample_paths.iterrows()):
            img_path = os.path.join(directory, f"{row[image_col]}.png")
            img = Image.open(img_path)
            plt.subplot(len(labels), samples_per_class, class_idx * samples_per_class + i + 1)
            plt.imshow(img)
            plt.axis('off')
            plt.title(f'Label: {label}')
    
    plt.tight_layout()
    plt.show()

show_sample_images(train_df, TRAIN_DIR)

In [4]:
# 4. Inspect one image shape

sample_img_path = os.path.join(TRAIN_DIR, train_df.iloc[0]['id_code'] + ".png")
sample_img = Image.open(sample_img_path)
print("Image size:", sample_img.size)
print("Image mode:", sample_img.mode)

Image size: (224, 224)
Image mode: RGB


In [5]:
# 5. Define ImageDataGenerators

train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    zoom_range=0.2,
    rotation_range=20,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
# 6. Add full file paths & convert labels to string for generators

train_df['file_path'] = train_df['id_code'].apply(lambda x: os.path.join(TRAIN_DIR, f"{x}.png"))
val_df['file_path'] = val_df['id_code'].apply(lambda x: os.path.join(VAL_DIR, f"{x}.png"))

train_df['diagnosis'] = train_df['diagnosis'].astype(str)
val_df['diagnosis'] = val_df['diagnosis'].astype(str)

In [7]:
# 7. Balance training data with oversampling minority classes

from sklearn.utils import resample

df_majority = train_df[train_df['diagnosis'] == "0"]
df_minority = train_df[train_df['diagnosis'] != "0"]

df_minority_upsampled = resample(df_minority,
                                 replace=True,
                                 n_samples=len(df_majority),
                                 random_state=SEED)

train_df_balanced = pd.concat([df_majority, df_minority_upsampled])
train_df_balanced = train_df_balanced.sample(frac=1, random_state=SEED).reset_index(drop=True)

# Convert diagnosis to string again (ensure correct type)
train_df_balanced['diagnosis'] = train_df_balanced['diagnosis'].astype(str)

In [8]:
# 8. Create data generators with balanced data

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df_balanced,
    x_col='file_path',
    y_col='diagnosis',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=SEED
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='file_path',
    y_col='diagnosis',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 2888 validated image filenames belonging to 5 classes.
Found 366 validated image filenames belonging to 5 classes.


In [9]:
# 9. Compute class weights (convert diagnosis to int temporarily)

from sklearn.utils.class_weight import compute_class_weight

train_df_int = train_df.copy()
train_df_int['diagnosis'] = train_df_int['diagnosis'].astype(int)

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_df_int['diagnosis']),
    y=train_df_int['diagnosis']
)

class_weights_dict = dict(enumerate(class_weights))
print("Class weights:", class_weights_dict)

Class weights: {0: 0.4056786703601108, 1: 1.979054054054054, 2: 0.7331664580725907, 3: 3.803896103896104, 4: 2.4822033898305085}


In [10]:
# 10. Import model and training utilities

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.optimizers import Adam 
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

/Users/nikhilgiji/Drive/cs/computer_vision/RetinoGrade/retino/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [11]:
# 11. Build and compile ResNet50 model

input_tensor = Input(shape=(*IMG_SIZE, 3))
base_model = ResNet50(include_top=False, weights='imagenet', input_tensor=input_tensor)

# Optionally freeze base model or train from start
base_model.trainable = True

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(5, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

2025-07-03 10:39:03.930785: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-07-03 10:39:03.930846: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-07-03 10:39:03.930856: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2025-07-03 10:39:03.930918: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-07-03 10:39:03.930943: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [12]:
# 12. Setup callbacks

checkpoint_cb = ModelCheckpoint("best_model.h5", save_best_only=True, monitor='val_loss', mode='min')
earlystop_cb = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr_cb = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

In [13]:
# 13. Define Macro F1 callback for validation metric

from sklearn.metrics import f1_score
import tensorflow as tf
import numpy as np

class MacroF1Callback(tf.keras.callbacks.Callback):
    def __init__(self, val_generator):
        super().__init__()
        self.val_generator = val_generator

    def on_epoch_end(self, epoch, logs=None):
        val_preds = []
        val_trues = []
        for i in range(len(self.val_generator)):
            x_val, y_val = self.val_generator[i]
            preds = self.model.predict(x_val)
            val_preds.extend(np.argmax(preds, axis=1))
            val_trues.extend(np.argmax(y_val, axis=1))
        f1 = f1_score(val_trues, val_preds, average='macro')
        print(f"\nEpoch {epoch+1} Macro F1: {f1:.4f}")

In [15]:
# 14. Train the model with class weights and callbacks

EPOCHS = 30
macro_f1_cb = MacroF1Callback(val_generator)

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    class_weight=class_weights_dict,
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr_cb, macro_f1_cb]
)

Epoch 1/30


2025-07-03 10:39:34.962032: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-07-03 10:39:35.523976: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


91/91 [==============================] - ETA: 0s - loss: 1.4635 - accuracy: 0.5772

/Users/nikhilgiji/Drive/cs/computer_vision/RetinoGrade/retino/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 1s 549ms/step

Epoch 1 Macro F1: 0.1319
91/91 [==============================] - 58s 519ms/step - loss: 1.4635 - accuracy: 0.5772 - val_loss: 6.8301 - val_accuracy: 0.4918 - lr: 1.0000e-04
Epoch 2/30
91/91 [==============================] - ETA: 0s - loss: 0.9300 - accuracy: 0.7064

...

1/1 [==============================] - 0s 27ms/step

Epoch 12 Macro F1: 0.4725
91/91 [==============================] - 43s 473ms/step - loss: 0.2096 - accuracy: 0.9387 - val_loss: 1.0763 - val_accuracy: 0.7268 - lr: 2.0000e-05
Epoch 13/30
1/1 [==============================] - 0s 19ms/step

Epoch 13 Macro F1: 0.5540
91/91 [==============================] - 961s 11s/step - loss: 0.1892 - accuracy: 0.9422 - val_loss: 1.1013 - val_accuracy: 0.7732 - lr: 2.0000e-05
Epoch 14/30
1/1 [==============================] - 0s 25ms/step

Epoch 14 Macro F1: 0.5824
91/91 [==============================] - 43s 469ms/step - loss: 0.2073 - accuracy: 0.9398 - val_loss: 1.2120 - val_accuracy: 0.7869 - lr: 2.0000e-05
Epoch 15/30
91/91 [==============================] - ETA: 0s - loss: 0.1750 - accuracy: 0.9491

/Users/nikhilgiji/Drive/cs/computer_vision/RetinoGrade/retino/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 29ms/step

Epoch 15 Macro F1: 0.6040
91/91 [==============================] - 673s 7s/step - loss: 0.1750 - accuracy: 0.9491 - val_loss: 0.9985 - val_accuracy: 0.7814 - lr: 2.0000e-05
Epoch 16/30
91/91 [==============================] - ETA: 0s - loss: 0.1551 - accuracy: 0.9557

/Users/nikhilgiji/Drive/cs/computer_vision/RetinoGrade/retino/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 21ms/step

Epoch 16 Macro F1: 0.6185
91/91 [==============================] - 43s 474ms/step - loss: 0.1551 - accuracy: 0.9557 - val_loss: 0.9874 - val_accuracy: 0.7978 - lr: 2.0000e-05
Epoch 17/30
1/1 [==============================] - 0s 19ms/step

Epoch 17 Macro F1: 0.6145
91/91 [==============================] - 43s 469ms/step - loss: 0.1504 - accuracy: 0.9574 - val_loss: 1.0971 - val_accuracy: 0.7951 - lr: 2.0000e-05
Epoch 18/30
1/1 [==============================] - 0s 27ms/step

Epoch 18 Macro F1: 0.5975
91/91 [==============================] - 91s 1s/step - loss: 0.1523 - accuracy: 0.9564 - val_loss: 1.0393 - val_accuracy: 0.7869 - lr: 2.0000e-05
Epoch 19/30
1/1 [==============================] - 0s 19ms/step

Epoch 19 Macro F1: 0.6022
91/91 [==============================] - 43s 467ms/step - loss: 0.1233 - accuracy: 0.9602 - val_loss: 1.1411 - val_accuracy: 0.7923 - lr: 2.0000e-05
Epoch 20/30
1/1 [==============================] - 0s 1

In [16]:
# 15. Validation evaluation: classification report & confusion matrix

from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

val_generator.reset()
preds = model.predict(val_generator, verbose=1)
y_pred = np.argmax(preds, axis=1)
y_true = val_generator.classes

print(classification_report(y_true, y_pred, target_names=list(val_generator.class_indices.keys())))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=val_generator.class_indices.keys(), 
            yticklabels=val_generator.class_indices.keys())
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show() 

12/12 [==============================] - 2s 132ms/step
              precision    recall  f1-score   support

           0       0.94      0.97      0.96       180
           1       0.52      0.62      0.57        37
           2       0.71      0.79      0.75       100
           3       0.67      0.30      0.41        20
           4       0.60      0.31      0.41        29

    accuracy                           0.80       366
   macro avg       0.69      0.60      0.62       366
weighted avg       0.79      0.80      0.79       366



In [17]:
# 16. Validation metrics: balanced accuracy, macro F1, AUC per class

from sklearn.metrics import balanced_accuracy_score, f1_score, roc_auc_score

bal_acc = balanced_accuracy_score(y_true, y_pred)
print(f"Balanced Accuracy: {bal_acc:.4f}")

macro_f1 = f1_score(y_true, y_pred, average='macro')
print(f"Macro F1-score: {macro_f1:.4f}")

try:
    aucs = []
    for i in range(5):
        auc = roc_auc_score((y_true == i).astype(int), preds[:, i])
        aucs.append(auc)
        print(f"AUC for class {i}: {auc:.4f}")
    print(f"Mean AUC: {np.mean(aucs):.4f}")
except Exception as e:
    print("Error calculating AUC:", e)

Balanced Accuracy: 0.5988
Macro F1-score: 0.6185
Error calculating AUC: 'bool' object has no attribute 'astype'


In [18]:
# 17. Threshold tuning on validation predictions (optional)

def apply_thresholds(probs, thresholds):
    preds = np.zeros_like(probs)
    for i, thr in enumerate(thresholds):
        preds[:, i] = (probs[:, i] >= thr).astype(int)
    return preds

thresholds = [0.5] * 5

pred_binary = apply_thresholds(preds, thresholds)

final_preds = []
for i in range(len(pred_binary)):
    if pred_binary[i].sum() == 0:
        final_preds.append(0)
    elif pred_binary[i].sum() == 1:
        final_preds.append(np.argmax(pred_binary[i]))
    else:
        candidates = np.where(pred_binary[i] == 1)[0]
        best = candidates[np.argmax(preds[i][candidates])]
        final_preds.append(best)

final_preds = np.array(final_preds)

from sklearn.metrics import classification_report

print(classification_report(y_true, final_preds, target_names=list(val_generator.class_indices.keys())))

              precision    recall  f1-score   support

           0       0.90      0.97      0.94       180
           1       0.52      0.59      0.56        37
           2       0.72      0.77      0.74       100
           3       0.67      0.30      0.41        20
           4       0.57      0.28      0.37        29

    accuracy                           0.79       366
   macro avg       0.68      0.58      0.60       366
weighted avg       0.77      0.79      0.77       366



In [20]:
# 18. Test data inference (using model in memory)

import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_df = pd.read_csv(TEST_CSV)
test_df['file_path'] = test_df['id_code'].apply(lambda x: os.path.join(TEST_DIR, f"{x}.png"))

if 'diagnosis' in test_df.columns:
    test_df['diagnosis'] = test_df['diagnosis'].astype(str)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='file_path',
    y_col='diagnosis' if 'diagnosis' in test_df.columns else None,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical' if 'diagnosis' in test_df.columns else None,
    shuffle=False
)

# Reset generator before prediction
test_generator.reset()
pred_probs = model.predict(test_generator, verbose=1)

# Convert predicted probabilities to class indices
pred_classes = pred_probs.argmax(axis=1)

# Map class indices back to labels
class_indices = test_generator.class_indices
inv_class_indices = {v: k for k, v in class_indices.items()}
pred_labels = [inv_class_indices[i] for i in pred_classes]

# If true labels available, evaluate performance
if 'diagnosis' in test_df.columns:
    from sklearn.metrics import classification_report, confusion_matrix
    import seaborn as sns
    import matplotlib.pyplot as plt

    y_true = test_generator.classes
    print(classification_report(y_true, pred_classes, target_names=list(class_indices.keys())))

    cm = confusion_matrix(y_true, pred_classes)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_indices.keys(), yticklabels=class_indices.keys())
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix on Test Set")
    plt.show()

# Save predictions to CSV
test_df['predicted_label'] = pred_labels
test_df[['id_code', 'predicted_label']].to_csv('test_predictions_resnet.csv', index=False)

Found 367 validated image filenames belonging to 5 classes.
12/12 [==============================] - 2s 167ms/step
              precision    recall  f1-score   support

           0       0.97      0.98      0.98       181
           1       0.67      0.59      0.63        37
           2       0.74      0.81      0.77       100
           3       0.35      0.37      0.36        19
           4       0.71      0.50      0.59        30

    accuracy                           0.83       367
   macro avg       0.69      0.65      0.67       367
weighted avg       0.82      0.83      0.82       367

